# Workflow for generating invariant cellular landmarks on spatial proteomics data
Settings are communicated via parameter files. These are python files usually containing a dictionary of settings that are used by the individual steps.
You can find a complete set of example parameter files in notebooks/params.

In the following we will use these parameter files to use the cVAE framework to generate ICLs for unperturbed and meayamycin perturbed cells.

## 1. Setup
TODO
- change config.ini
- create constants.py + explain it
- download data

The data is represented as an MPPData object. For more information on this class and the data representation on disk see [the tutorial on MPPData](mpp_data.ipynb).

TODO mention ImageData?

## 2. Create NNDataset
NNDatasets are created using a data params file specifying the data, normalisation, and subsampling for generating a NNDataset. 
Internally, an NNDataset represents train, val and test splits as MPPData objects. For more infomation on NNDataset, see [the tutorial on creating and working with a dataset using NNDataset](nn_dataset.ipynb)

The NNDataset can easily be created with the cli:
`python create_dataset.py params/example_data_params.py`

## 3. Train cVAE
For training and evaluation, an experiment_params.py file is needed.
This file defines several experiments. For more information on the training and evaluation process, see [the tutorial on training cVAE models](train.ipynb)

To train, evaluate and cluster models, run:
`python train.py all --config params/example_experiment_params.py`

## 4. Cluster


## 5. Extract features